In [18]:
#import pymeshlab

# Imports

In [17]:
import subprocess, re, os, sys
import matplotlib.pyplot as plt
import nibabel as nib
import os
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import pyvista
from skimage.measure import marching_cubes
import numpy as np
import shutil
from itertools import permutations
import json
#import open3d as o3d

In [2]:
import open3d as o3d

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
# GPU picking
# http://stackoverflow.com/a/41638727/419116
# Nvidia-smi GPU memory parsing.
# Tested on nvidia-smi 370.23

def run_command(cmd):
    """Run command, return output as string."""
    
    output = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True).communicate()[0]
    return output.decode("ascii")

def list_available_gpus():
    """Returns list of available GPU ids."""
    
    output = run_command("nvidia-smi -L")
    # lines of the form GPU 0: TITAN X
    gpu_regex = re.compile(r"GPU (?P<gpu_id>\d+):")
    result = []
    for line in output.strip().split("\n"):
        m = gpu_regex.match(line)
        assert m, "Couldnt parse "+line
        result.append(int(m.group("gpu_id")))
    return result

def gpu_memory_map():
    """Returns map of GPU id to memory allocated on that GPU."""

    output = run_command("nvidia-smi")
    gpu_output = output[output.find("GPU Memory"):]
    # lines of the form
    # |    0      8734    C   python                                       11705MiB |
    memory_regex = re.compile(r"[|]\s+?(?P<gpu_id>\d+)\D+?(?P<pid>\d+).+[ ](?P<gpu_memory>\d+)MiB")
    rows = gpu_output.split("\n")
    result = {gpu_id: 0 for gpu_id in list_available_gpus()}
    for row in gpu_output.split("\n"):
        m = memory_regex.search(row)
        if not m:
            continue
        gpu_id = int(m.group("gpu_id"))
        gpu_memory = int(m.group("gpu_memory"))
        result[gpu_id] += gpu_memory
    return result

def pick_gpu_lowest_memory():
    """Returns GPU with the least allocated memory"""

    memory_gpu_map = [(memory, gpu_id) for (gpu_id, memory) in gpu_memory_map().items()]
    best_memory, best_gpu = sorted(memory_gpu_map)[0]
    return best_gpu

def setup_one_gpu():
    assert not 'tensorflow' in sys.modules, "GPU setup must happen before importing TensorFlow"
    gpu_id = pick_gpu_lowest_memory()
    print("Picking GPU "+str(gpu_id))
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

def setup_no_gpu():
    if 'tensorflow' in sys.modules:
        print("Warning, GPU setup must happen before importing TensorFlow")
    os.environ["CUDA_VISIBLE_DEVICES"] = ''

In [5]:
setup_one_gpu()

Picking GPU 1


# Functions

In [23]:
def pad_edge_list(edges):
    padding = np.ones(edges.shape[0], int)*3
    edges_w_padding = np.vstack((padding, edges.T)).T
    return edges_w_padding

def organs_data(abdominal_segmentations_data, dir):
    try:
        verts1, faces1, norms1, vals1 = marching_cubes(abdominal_segmentations_data==1, level=0, step_size=1)
        #verts2, faces2, norms2, vals2 = marching_cubes(abdominal_segmentations_data==2, level=0, step_size=1)
        #verts3, faces3, norms3, vals3 = marching_cubes(abdominal_segmentations_data==3, level=0, step_size=1)
        #verts4, faces4, norms4, vals4 = marching_cubes(abdominal_segmentations_data==4, level=0, step_size=1)
        #verts5, faces5, norms5, vals5 = marching_cubes(abdominal_segmentations_data==5, level=0, step_size=1)
        verts1 = verts1/np.array(abdominal_segmentations_data.shape) 
        #verts2 = verts2/np.array(abdominal_segmentations_data.shape) 
        #verts3 = verts3/np.array(abdominal_segmentations_data.shape) 
        #verts4 = verts4/np.array(abdominal_segmentations_data.shape) # to normalize ponit coordinate in [0,1]
        #verts5 = verts5/np.array(abdominal_segmentations_data.shape) # to normalize ponit coordinate in [0,1]
        edges1 = np.concatenate((faces1[:,:2], faces1[:,1:]), axis=0)
        #edges2 = np.concatenate((faces2[:,:2], faces2[:,1:]), axis=0)
        #edges3 = np.concatenate((faces3[:,:2], faces3[:,1:]), axis=0)
        #edges4 = np.concatenate((faces4[:,:2], faces4[:,1:]), axis=0)
        #edges5 = np.concatenate((faces5[:,:2], faces5[:,1:]), axis=0)

        lines1 = np.concatenate((np.int32(2*np.ones((edges1.shape[0],1))), edges1), 1)
        #lines2 = np.concatenate((np.int32(2*np.ones((edges2.shape[0],1))), edges2), 1)
        #lines3 = np.concatenate((np.int32(2*np.ones((edges3.shape[0],1))), edges3), 1)
        #lines4 = np.concatenate((np.int32(2*np.ones((edges4.shape[0],1))), edges4), 1)
        #lines5 = np.concatenate((np.int32(2*np.ones((edges5.shape[0],1))), edges5), 1)
        mesh1 = pyvista.PolyData(verts1, pad_edge_list(faces1))
        #mesh2 = pyvista.PolyData(verts2, pad_edge_list(faces2))
        #mesh3 = pyvista.PolyData(verts3, pad_edge_list(faces3))
        #mesh4 = pyvista.PolyData(verts4, pad_edge_list(faces4))
        #mesh5 = pyvista.PolyData(verts5, pad_edge_list(faces5))

        mesh1.lines = lines1.flatten()
        #mesh2.lines = lines2.flatten()
        #mesh3.lines = lines3.flatten()
        #mesh4.lines = lines4.flatten()
        #mesh5.lines = lines5.flatten()

        # verts = [verts1, verts2, verts3, verts4, verts5]
        # edges = [edges1, edges2, edges3, edges4, edges5]
        # faces = [faces1, faces2, faces3, faces4, faces5]
        # lines = [lines1, lines2, lines3, lines4, lines5]
        # meshes = [mesh1, mesh2, mesh3, mesh4, mesh5]
        # norms = [norms1, norms2, norms3, norms4, norms5]
        # vals = [vals1, vals2, vals3, vals4, vals5]

        verts = [verts1]
        edges = [edges1]
        faces = [faces1]
        lines = [lines1]
        meshes = [mesh1]
        norms = [norms1]
        vals = [vals1]

        data = [verts, edges, faces, lines, meshes, norms, vals]
        return data
    except:
        x=0

def triangle_mesh(verts, faces):
    mesh_list = []
    for i in range(0, len(verts)):
        mesh_list.append(o3d.geometry.TriangleMesh(vertices = o3d.utility.Vector3dVector(np.asarray(verts[i])), triangles = o3d.utility.Vector3iVector(np.asarray(faces[i]))))
        #mesh.compute_vertex_normals()
        #mesh.paint_uniform_color([0, 1, 0])
        #vis = o3d.visualization.Visualizer()
        #vis.create_window()
        #vis.add_geometry(mesh)
        #vis.update_geometry()
        #vis.poll_events()
        #vis.update_renderer()
        #path = str(i) + ".jpg"
        #vis.capture_screen_image(path)
        #vis.destroy_window()

    return mesh_list

def get_min_point_triangle():
    min_vertices = [np.nan, np.nan, np.nan, np.nan, np.nan]
    min_triangles = [np.nan, np.nan, np.nan, np.nan, np.nan]
    sum_vertices = [0,0,0,0,0]
    sum_triangles = [0,0,0,0,0]
    count = [0,0,0,0,0]
    min_vertices_mesh = [np.nan, np.nan, np.nan, np.nan, np.nan]
    min_triangles_mesh = [np.nan, np.nan, np.nan, np.nan, np.nan]
    empty_files = []
    bad_qual = []
    path_to_segmentations = "organ_segmentations"
    for _root, dirs, _file_list in os.walk(path_to_segmentations):
        for dir in dirs:
            if(os.path.exists(os.path.join(path_to_segmentations, dir, "prd.nii.gz"))):
                abdominal_segmentations = nib.load(os.path.join(path_to_segmentations, dir, "prd.nii.gz"))
                abdominal_segmentations_data = abdominal_segmentations.get_fdata()

                data = organs_data(abdominal_segmentations_data, dir)
                if(data is not None):
                    mesh_list = triangle_mesh(data[0], data[2])
                    
                    for i in range(0, len(mesh_list)):
                        sum_vertices[i] = sum_vertices[i] + len(np.asarray(mesh_list[i].vertices)) 
                        sum_triangles[i] = sum_triangles[i] + len(np.asarray(mesh_list[i].triangles))
                        count[i] = count[i] + 1 
                        if(len(np.asarray(mesh_list[i].vertices)) < 2000):
                            print("Less than 2000 points: {}".format(dir))
                            bad_qual.append(dir)
                        else:
                            if(np.isnan(min_triangles[i])):
                                min_triangles[i] = len(np.asarray(mesh_list[i].triangles))
                                min_triangles_mesh[i] = dir
                            if(len(np.asarray(mesh_list[i].triangles)) < min_triangles[i]):
                                min_triangles[i] = len(np.asarray(mesh_list[i].triangles))
                                min_triangles_mesh[i] = dir

                            if(np.isnan(min_vertices[i])):
                                min_vertices[i] = len(np.asarray(mesh_list[i].vertices))
                                min_vertices_mesh[i] = dir
                            if(len(np.asarray(mesh_list[i].vertices)) < min_vertices[i]):
                                min_vertices[i] = len(np.asarray(mesh_list[i].vertices))
                                min_vertices_mesh[i] = dir
            else:
                empty_files.append(dir)

    minimums = [min_vertices, min_triangles]
    vert_avg = [i / j for i, j in zip(sum_vertices, count)]
    triangle_avg = [[i / j for i, j in zip(sum_triangles, count)]]
    averages = [vert_avg, triangle_avg]
    files = [min_vertices_mesh, min_triangles_mesh]
    return files, minimums, averages, empty_files, bad_qual
            
    
def dec_triangle_mesh(dec):
    path_to_segmentations = "organ_segmentations"
    for _root, dirs, _file_list in os.walk(path_to_segmentations):
        for dir in dirs:
            if(not os.path.exists(os.path.join("./organ_decimations_off", str(dir) , "liver_mesh.off"))):
                if(os.path.exists(os.path.join(path_to_segmentations, dir, "prd.nii.gz"))):
                    abdominal_segmentations = nib.load(os.path.join(path_to_segmentations, dir, "prd.nii.gz"))
                    abdominal_segmentations_data = abdominal_segmentations.get_fdata()

                    data = organs_data(abdominal_segmentations_data, dir)
                    if(data is not None):
                        mesh_list = triangle_mesh(data[0], data[2])

                        for i in range(0, len(mesh_list)):
                            dec_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh_list[i], dec)
                            dec_mesh_path = os.path.join("./organ_decimations_off", dir, "liver_mesh.off")
                            if(not os.path.exists(os.path.join("./organ_decimations_off", str(dir)))):
                                os.mkdir("./organ_decimations_off/" + str(dir))
                            o3d.io.write_triangle_mesh(dec_mesh_path, dec_mesh, print_progress=True)
    
    #for i in range(0, len(verts)):
    #    mesh = o3d.geometry.TriangleMesh(vertices = o3d.utility.Vector3dVector(np.asarray(verts[i])), triangles = o3d.utility.Vector3iVector(np.asarray(faces[i])))
    #    dec_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh, dec)
    #dec_mesh.compute_vertex_normals()
    #dec_mesh.paint_uniform_color([0, 1, 0])
    #vis = o3d.visualization.Visualizer()
    #vis.create_window()
    #vis.add_geometry(dec_mesh)
    # vis.update_geometry()
    #vis.poll_events()
    #vis.update_renderer()
    #path = "dec_" + str(dec) + "_" + str(i) + ".jpg"
    #vis.capture_screen_image(path)
    #vis.destroy_window()
    # return mesh_list, dec_mesh_list

def off_to_json():
    path = "organ_decimations_off"
    for _root, dirs, _file_list in os.walk(path):
        for dir in dirs:
            if(os.path.exists(os.path.join(path, dir, "liver_mesh.off"))):
                if(not os.path.exists(os.path.join("./organ_decimations_json", str(dir) , "liver_mesh.off"))):
                    file = open("./organ_decimations_off/" + str(dir) + "/liver_mesh.off", "r")
                    lines_list = file.readlines()
                    file_info = lines_list[1].split(" ")
                    points_list = lines_list[2:(int(file_info[0])+2)]
                    faces_list = lines_list[(int(file_info[0])+2):]

                    split_points_list = []
                    for point in points_list:
                        new_structure = point.split(" ")
                        new_structure[2] = new_structure[2][:-1]
                        split_points_list.append(new_structure)

                    split_faces_list = []
                    for face in faces_list:
                        new_structure = face.split(" ")[1:]
                        new_structure[2] = new_structure[2][:-1]
                        split_faces_list.append(new_structure)

                    edges = []
                    for face in split_faces_list:
                        comb = permutations(face, 2)
                        for i in list(comb):
                            edges.append(i)

                    json_data = {
                        "data_points": split_points_list,
                        "edges": edges
                    }

                    json_file = json.dumps(json_data)
                    if(not os.path.exists(os.path.join("./organ_decimations_json", str(dir)))):
                        os.mkdir("./organ_decimations_json/" + str(dir))
                    with open("./organ_decimations_json/" + str(dir) + "/liver_mesh.json", "w") as outfile:
                        outfile.write(json_file)
                    shutil.rmtree(os.path.join("./organ_decimations_off", str(dir)))
    print("Done")


# Pyvista

In [21]:
path_to_segmentations = "organ_segmentations"
subj_id = 6019150
abdominal_segmentations = nib.load(os.path.join(path_to_segmentations, str(subj_id), "prd.nii.gz"))
abdominal_segmentations_data = abdominal_segmentations.get_fdata()

verts1, faces1, norms1, vals1 = marching_cubes(abdominal_segmentations_data==1, level=0, step_size=1)
verts2, faces2, norms2, vals2 = marching_cubes(abdominal_segmentations_data==2, level=0, step_size=1)
verts3, faces3, norms3, vals3 = marching_cubes(abdominal_segmentations_data==3, level=0, step_size=1)
verts4, faces4, norms4, vals4 = marching_cubes(abdominal_segmentations_data==4, level=0, step_size=1)
verts5, faces5, norms5, vals5 = marching_cubes(abdominal_segmentations_data==5, level=0, step_size=1)
verts1 = verts1/np.array(abdominal_segmentations_data.shape) 
verts2 = verts2/np.array(abdominal_segmentations_data.shape) 
verts3 = verts3/np.array(abdominal_segmentations_data.shape) 
verts4 = verts4/np.array(abdominal_segmentations_data.shape) # to normalize ponit coordinate in [0,1]
verts5 = verts5/np.array(abdominal_segmentations_data.shape) # to normalize ponit coordinate in [0,1]
edges1 = np.concatenate((faces1[:,:2], faces1[:,1:]), axis=0)
edges2 = np.concatenate((faces2[:,:2], faces2[:,1:]), axis=0)
edges3 = np.concatenate((faces3[:,:2], faces3[:,1:]), axis=0)
edges4 = np.concatenate((faces4[:,:2], faces4[:,1:]), axis=0)
edges5 = np.concatenate((faces5[:,:2], faces5[:,1:]), axis=0)

pyvista.set_plot_theme("document")
plotter = pyvista.Plotter(shape=(1, 1), window_size=[1000, 500], border=False)
plotter.subplot(0, 0)
lines1 = np.concatenate((np.int32(2*np.ones((edges1.shape[0],1))), edges1), 1)
lines2 = np.concatenate((np.int32(2*np.ones((edges2.shape[0],1))), edges2), 1)
lines3 = np.concatenate((np.int32(2*np.ones((edges3.shape[0],1))), edges3), 1)
lines4 = np.concatenate((np.int32(2*np.ones((edges4.shape[0],1))), edges4), 1)
lines5 = np.concatenate((np.int32(2*np.ones((edges5.shape[0],1))), edges5), 1)
mesh1 = pyvista.PolyData(verts1, pad_edge_list(faces1))
mesh2 = pyvista.PolyData(verts2, pad_edge_list(faces2))
mesh3 = pyvista.PolyData(verts3, pad_edge_list(faces3))
mesh4 = pyvista.PolyData(verts4, pad_edge_list(faces4))
mesh5 = pyvista.PolyData(verts5, pad_edge_list(faces5))

mesh1.lines = lines1.flatten()
mesh2.lines = lines2.flatten()
mesh3.lines = lines3.flatten()
mesh4.lines = lines4.flatten()
mesh5.lines = lines5.flatten()
plotter.add_mesh(mesh1, render_points_as_spheres=False, color="lightcoral", show_edges=True, line_width=1, edge_color='mediumorchid', point_size=2, )
plotter.add_mesh(mesh2, render_points_as_spheres=False, color="#ad4dff", show_edges=True, line_width=1, edge_color='mediumorchid', point_size=0.4)
plotter.add_mesh(mesh3, render_points_as_spheres=False, color="#409aff", show_edges=True, line_width=1, edge_color='mediumorchid', point_size=0.4)
plotter.add_mesh(mesh4, render_points_as_spheres=False, color="#409aff", show_edges=True, line_width=1, edge_color='mediumorchid', point_size=0.4)
plotter.add_mesh(mesh5, render_points_as_spheres=False, color="#ffcd00", show_edges=True, line_width=1, edge_color='mediumorchid', point_size=0.4)

plotter.camera.zoom(7)
# plotter.fly_to((0,-0.6, 0))
plotter.set_position((0,-5,2))
plotter.save_graphic("organ_meshes.svg")  
#plotter.show()


verts = [verts1, verts2, verts3, verts4, verts5]
edges = [edges1, edges2, edges3, edges4, edges5]
faces = [faces1, faces2, faces3, faces4, faces5]
lines = [lines1, lines2, lines3, lines4, lines5]
meshes = [mesh1, mesh2, mesh3, mesh4, mesh5]
norms = [norms1, norms2, norms3, norms4, norms5]
vals = [vals1, vals2, vals3, vals4, vals5]

mesh = o3d.geometry.TriangleMesh(vertices = o3d.utility.Vector3dVector(np.asarray(verts2)), triangles = o3d.utility.Vector3iVector(np.asarray(faces2)))
mesh

TriangleMesh with 4550 points and 9096 triangles.

# open3D

In [5]:
# with open3d: point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.asarray(verts1))

o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [6]:
# with open3d: graph
lines = o3d.utility.Vector2iVector(np.asarray(edges1))
line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.asarray(verts1)),
        lines=lines)

line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for i in range(len(lines1))])

o3d.visualization.draw_geometries([line_set])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [15]:
# with open3d: triangle mesh
mesh = o3d.geometry.TriangleMesh(vertices = o3d.utility.Vector3dVector(np.asarray(verts2)), triangles = o3d.utility.Vector3iVector(np.asarray(faces2)))
#mesh.compute_vertex_normals()
#mesh.paint_uniform_color([0, 1, 0]) 
#vis = o3d.visualization.Visualizer()
#vis.create_window()
#vis.add_geometry(mesh)
# vis.update_geometry()
#vis.poll_events()
#vis.update_renderer()
#path = "liver_mesh.jpg"
#vis.capture_screen_image(path)
#vis.destroy_window()
mesh
# o3d.visualization.draw_geometries([mesh])

TriangleMesh with 7904 points and 15804 triangles.

In [24]:
len(np.asarray(mesh.triangles))

39076

In [23]:
len(np.asarray(mesh.vertices))

19540

# Decimation

In [38]:
files, minimums, averages, empty_files, bad_qual = get_min_point_triangle()

Less than 2000 points: 3849529
Less than 2000 points: 4602530
Less than 2000 points: 5283486
Less than 2000 points: 3271458
Less than 2000 points: 5870022
Less than 2000 points: 1286466
Less than 2000 points: 1286466
Less than 2000 points: 5631928
Less than 2000 points: 3469204
Less than 2000 points: 1881877
Less than 2000 points: 1030350
Less than 2000 points: 5559180
Less than 2000 points: 3633588
Less than 2000 points: 5353750
Less than 2000 points: 5312364
Less than 2000 points: 3734888
Less than 2000 points: 5218617
Less than 2000 points: 5660586
Less than 2000 points: 4293892
Less than 2000 points: 3618196
Less than 2000 points: 2009066
Less than 2000 points: 1361112
Less than 2000 points: 2659432
Less than 2000 points: 1134408
Less than 2000 points: 5542421
Less than 2000 points: 3116399
Less than 2000 points: 5554979
Less than 2000 points: 5613423
Less than 2000 points: 5785102
Less than 2000 points: 5298476
Less than 2000 points: 2448916
Less than 2000 points: 3141459
Less tha

In [ ]:
len(empty_files)

13352

In [ ]:
files

In [ ]:
minimums

[[2488, 6, 6, 22, 38], [4974, 8, 8, 40, 72]]

In [ ]:
averages

In [ ]:
bad_qual

In [ ]:
len(bad_qual)

In [20]:
mlist = triangle_mesh(verts, faces)
mlist

[TriangleMesh with 17364 points and 34728 triangles.,
 TriangleMesh with 3094 points and 6184 triangles.,
 TriangleMesh with 4228 points and 8452 triangles.,
 TriangleMesh with 3822 points and 7640 triangles.,
 TriangleMesh with 3334 points and 6664 triangles.]

In [29]:
mesh = o3d.geometry.TriangleMesh(vertices = o3d.utility.Vector3dVector(np.asarray(verts1)), triangles = o3d.utility.Vector3iVector(np.asarray(faces1)))
dec_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh, 4974)
#dec_mesh.compute_vertex_normals()
#dec_mesh.paint_uniform_color([0, 1, 0])
#vis = o3d.visualization.Visualizer()
#vis.create_window()
#vis.add_geometry(dec_mesh)
# vis.update_geometry()
#vis.poll_events()
#vis.update_renderer()
#path = "liver_20000_dec_mesh.jpg"
#vis.capture_screen_image(path)
#vis.destroy_window()
dec_mesh

TriangleMesh with 2573 points and 4973 triangles.

In [61]:
o3d.visualization.draw_geometries([dec_mesh])

In [12]:
dec_triangle_mesh(2000)

Writing OFF: [========================================] 100%


In [24]:
off_to_json()

# Visuals for all organs

In [34]:
triangle_mesh(verts, faces)

In [36]:
dec_triangle_mesh(verts, faces, 4000)

In [4]:
print(len(next(os.walk('organ_segmentations'))[1]))

44205


In [15]:
print(len(next(os.walk('organ_decimations_off'))[1]))

30721


In [ ]:
print(len(next(os.walk('organ_decimations_json'))[1]))